# Cookbook

We start with importing the CrossPy library, as well as NumPy and CuPy for demostration.

In [5]:
import crosspy as xp
import numpy as np
import cupy as cp

## Create arrays

We create a NumPy array and a CuPy and then merge them into a CrossPy array.

In [6]:
x_cpu = np.array([1, 2, 3])
x_cpu

array([1, 2, 3])

In [7]:

x_gpu = cp.array([4, 5])
x_gpu

array([4, 5])

In [8]:
x_gpu.device

<CUDA Device 0>

In [9]:
x_cross = xp.array([x_cpu, x_gpu], axis=0)
x_cross

array {((0, 3),): array([1, 2, 3]), ((3, 5),): array([4, 5])}

In [10]:
x_cpu.shape

(3,)

In [11]:
x_gpu.shape

(2,)

In [12]:
x_cross.shape

(5,)

In [13]:
x = xp.array([x_gpu, x_gpu])
x

array {((0, 2), (0, 2)): [array([4, 5]), array([4, 5])]}

In [14]:
x.shape

(2, 2)

In [15]:
x = xp.array([x_gpu, x_gpu], axis=0)
x

array {((0, 2),): array([4, 5]), ((2, 4),): array([4, 5])}

In [16]:
x.shape

(4,)

In [17]:
# xp.array([x_cpu, x_cpu.T])

## Indexing

In [18]:
x = x_cross[0] # slicing with a single integer
x

1

In [19]:
x.shape

()

In [20]:
x = x_cross[[0, 3]] # slicing with a list of integers
x

array {((0, 1),): array([1]), ((1, 2),): array([5])}

In [21]:
x.shape

(2,)

In [22]:
# x = x_cross[0:2] # slicing with Python built-in slices
# x

AssertionError: assumption: no concat for non-list objs

In [ ]:
x.shape

In [ ]:
from crosspy import cpu, gpu

In [ ]:
# a = xp.array(range(6), distribution=[cpu(0), gpu(0), gpu(1)])
# a

In [ ]:
# a.device

In [ ]:
# from crosspy import PartitionScheme
# partition = PartitionScheme(6, default_device=cpu(0))

# partition[(0, 4, 5)] = cpu(0)
# partition[1:3] = gpu(0)
# partition[3] = gpu(1)

# a = xp.array(range(6), distribution=partition)
# a

In [ ]:
# a.device

## Arithmetics

In [ ]:
# a[0] = a[2] + a[4]
# a

## Interoperability with NumPy/CuPy


CrossPy arrays can be assigned value(s) from/to NumPy/CuPy arrays.

When assigning values from NumPy/CuPy arrays to CrossPy arrays, there are two
possible behaviors. The first one scatters the data from the source array to the
underlying devices of the CrossPy array, i.e., the heterogeneity of the CrossPy
array is unchanged. The second one overwrites the corresponding part of the
target array with both the data and the device of the source array. The built-in
assignment operation in Python is not overloadable and we chose to implement it
with the scatter behavior.

>>> x_cross[0] = np.array([5, 4, 3, 2, 1])
>>> x_cross # doctest: +NORMALIZE_WHITESPACE
array {((0, 1), (0, 3)): array([[5, 4, 3]]),
       ((0, 1), (3, 5)): array([[2, 1]])}
>>> x_cross[0] = cp.array([6, 7, 8, 9, 0])
>>> x_cross # doctest: +NORMALIZE_WHITESPACE
array {((0, 1), (0, 3)): array([[6, 7, 8]]),
       ((0, 1), (3, 5)): array([[9, 0]])}

For assigning values from CrossPy arrays to NumPy/CuPy arrays, since the target
is distinguishable by devices (NumPy arrays are always on CPU while CuPy arrays are
on GPU devices), we use ``to`` to convert the array. We simply use negative integers
as CPU devices and otherwise GPU devices.

>>> y_cpu = x_cross.to(-1)
>>> y_cpu
array([[6, 7, 8, 9, 0]])
>>> type(y_cpu)
<class 'numpy.ndarray'>

>>> y_gpu0 = x_cross[:1, (0, 2, 4)].to(0)
>>> y_gpu0
array([[6, 8, 0]])
>>> type(y_gpu0)
<class 'cupy.core.core.ndarray'>
>>> y_gpu0.device
<CUDA Device 0>

>>> y_gpu1 = x_cross.to(1)
>>> y_gpu1
array([[6, 7, 8, 9, 0]])
>>> type(y_gpu1)
<class 'cupy.core.core.ndarray'>
>>> y_gpu1.device
<CUDA Device 1>

With ``to``, we can use NumPy/CuPy computational functions as usual.

>>> np.linalg.norm(x_cross.to(-1))
15.165750888103101
>>> cp.linalg.norm(x_cross.to(0))
array(15.16575089)

.. note::
    It seems tedious to have the ugly tail ``to``. However, third-party APIs always
    have fixed signatures and those in CuPy for example is inherently not compatible
    with CrossPy objects (same with NumPy). Therefore, an explicit operation is
    necessary to satisfy the input requirements of third-party APIs.